In [1]:
from fractalai.model import RandomDiscreteModel
from fractalai.environment import ExternalProcess, ParallelEnvironment, AtariEnvironment
from fractalai.swarm_wave import SwarmWave
import time

In [2]:
# SwarmWave?

# This algorithhm is made for solving discrete Markov decision processes when a perfect model is available

In this context, if you need to generate data to train a neural network, IO is now your bottleneck.

In [3]:
name = "MsPacman-ram-v0"
skip_frames = 0  # Number of frames to skip at the beginning
dt_mean = 4  # Apply the same action n times in average
dt_std = 2 # Repeat same action a variable number of times
min_dt = 3 # Minimum number of consecutive steps to be taken
n_samples = None  # Maximum number of samples allowed
reward_limit = 10000  # Stop the sampling when this score is reached
n_walkers = 250  # Maximum witdh of the tree containing al the trajectories
render_every = 1  # print statistics every n iterations.
balance = 1  # Balance exploration vs exploitation
save_data = True # Save the data generated


In [4]:
env = ParallelEnvironment(name=name,env_class=AtariEnvironment,
                          blocking=False, n_workers=8)  # We will play an Atari game
model = RandomDiscreteModel(max_wakers=n_walkers,
                            n_actions=env.n_actions) # The Agent will take discrete actions at random

In [5]:
%%time
s = SwarmWave(model=model, env=env, n_walkers=n_walkers, reward_limit=reward_limit, 
              render_every=render_every, balance=balance, save_data=save_data,
              dt_mean=dt_mean, dt_std=dt_std, accumulate_rewards=True, min_dt=min_dt)
s.run_swarm(print_swarm=True)


Environment: MsPacman-ram-v0 | Walkers: 250 | Deaths: 0 | data_size 408
Total samples: 393884 Progress: 100.11%
Reward: mean 9577.57 | Dispersion: 481.00 | max 10011.00 | min 9530.00 | std 43.67
Episode length: mean 1702.92 | Dispersion 50.00 | max 1721.00 | min 1671.00 | std 12.78
Dt: mean 4.15 | Dispersion: 5.00 | max 8.00 | min 3.00 | std 1.35
Status: Score limit reached.
Efficiency 0.40%
Generated 1580 Examples | 249.29 samples per example.

CPU times: user 7.66 s, sys: 202 ms, total: 7.87 s
Wall time: 42.4 s


In [6]:
s.render_game()